### Number of Total Files Needed

In [ ]:
import json
from collections import defaultdict
import os
import json
from pathlib import Path
from typing import Dict, List, Tuple
from collections import defaultdict

REQUESTED_DOCUMENTS = "../requested_documents.json"

# Load the requested documents
with open(REQUESTED_DOCUMENTS, 'r') as f:
    requested_docs = json.load(f)

# Initialize a dictionary to store cleaned documents
cleaned_docs = defaultdict(list)

total_files = 0

# Iterate through the requested documents
for k,v in requested_docs.items():
    for docs in v:
        if "transaction" in docs.lower() or "annual" in docs.lower():
            cleaned_docs[k].append(docs)
            total_files += 1

print("Total Files: ", total_files)


In [ ]:
ra_collection_path = "../RA_collection"
TARGET_FOLDER_MAPPING_PATH = './JSON_FILES/target_folder_mapping.json'

def scan_ra_collection_for_target_folders(ra_collection_path: Path) -> Dict[str, List]:
    """Scan RA_collection and create a mapping of target folders to file counts.
    
    Structure expected:
    RA_collection/
        {person_folder}/
            {page_folder}/
                {target_folder}/
                    file1.pdf
                    file2.pdf
    
    Args:
        ra_collection_path: Path to the RA_collection directory
        
    Returns:
        Dictionary mapping target_folder_name -> [file_count, page_folder_name]
    """
    
    target_folder_mapping = {}
    total_student_files = 0
    
    # Check if directory exists
    if not ra_collection_path.exists():
        print(f"❌ Error: Directory not found: {ra_collection_path}")
        return {}
    
    # Iterate through person folders (top-level directories)
    for person_folder in ra_collection_path.iterdir():
        # Skip if not a directory or if it's a hidden folder
        if not person_folder.is_dir() or person_folder.name.startswith('.'):
            continue
        
        person_name = person_folder.name
        print(f"\n📂 Scanning person folder: {person_name}")
        
        # Iterate through page folders (second level)
        for page_folder in person_folder.iterdir():
            # Skip if not a directory
            if not page_folder.is_dir() or page_folder.name.startswith('.'):
                continue
            
            page_folder_name = page_folder.name
            print(f"   📄 Page folder: {page_folder_name}")
            
            # Iterate through target folders (third level - these are the keys we want)
            for target_folder in page_folder.iterdir():
                # Skip if not a directory
                if not target_folder.is_dir() or target_folder.name.startswith('.'):
                    continue
                
                target_folder_name = target_folder.name
                
                # Count PDF files in this target folder
                pdf_count = 0
                for file in target_folder.iterdir():
                    if file.is_file() and file.name.lower().endswith('.pdf'):
                        if "278" in file.name.lower():
                            print(file.name)
                            pdf_count += 1
                            total_student_files += 1
                # Store in mapping: target_folder -> [file_count, page_folder]
                if target_folder_name in target_folder_mapping:
                    # If target folder already exists (same person appears in multiple pages)
                    # Add the counts
                    existing_count = target_folder_mapping[target_folder_name][0]
                    existing_page = target_folder_mapping[target_folder_name][1]
                    target_folder_mapping[target_folder_name] = [
                        existing_count + pdf_count,
                        f"{existing_page}, {page_folder_name}"
                    ]
                    print(f"      🔄 Updated: {target_folder_name} -> {pdf_count} PDFs (duplicate)")
                else:
                    target_folder_mapping[target_folder_name] = [pdf_count, page_folder_name]
                    print(f"      ✓ {target_folder_name} -> {pdf_count} PDFs")
    
    return target_folder_mapping, total_transaction_files

In [ ]:
# Gives the target folder mapping with the keys starting from the person name. Cleaned the key up basically.
def _clean_target_folder_mapping(target_folder_mapping):

    #We will clean the keys. We need to capture the key as follows
    #iterate through the key until "page" is seen. Then find the first "-" sign after "page". Then take the part after the "-" sign.

    res = {}
    for k,v in target_folder_mapping.items():
        #iterate through the key until "page" is seen. Then find the first "-" sign. Then take the part after the "-" sign.
        k = k.split("Page")
        if len(k) > 1:
            k = k[1]
        else:
            k = k[0]

        #if k has " - " then take the part after the "-" sign.
        if "-" in k:
            k = k.split("-")[1]
        else:
            k = k

        #remove all spaces from k
        k = k.replace(" ", "")
        k = k.lower()
        k = k.split(",")[0]
        k = k.replace(" ", "")
        k = k.lower()
        res[k] = v

    print("--------------------------------cleaned target folder mapping--------------------------------")
    return res

In [ ]:
STUDENT_REQS_DOCS = "../AutomationComparison/target_folder_mapping.json"
with open(STUDENT_REQS_DOCS, 'r') as f:
    student_reqs_docs = json.load(f)

student_reqs_docs_notcleaned, student_total_files = scan_ra_collection_for_target_folders(Path(ra_collection_path))

In [ ]:
student_reqs_docs = _clean_target_folder_mapping(student_reqs_docs_notcleaned)

print("Total files in OGE: ", total_files)
print("total files by students: ", student_total_files)


In [ ]:
for k,v in student_reqs_docs_notcleaned.items():
    print(k, v)